In [1]:
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import mglearn
import math
import random
import networkx as nx
from sympy import *      
from mpl_toolkits.mplot3d import Axes3D
import numpy as np        
from matplotlib import cm
import pylab
import time

In [3]:
## 전역변수 지정
#  A_state, A_node, A_edge(연결수), MAX, MIN, A, A_edges(연결튜플)
#  B_state, B_node, B_edge, inter_edges, AB_edges, AB_neighbor, B, B_edges
#  r, D, repeating_number, limited_time, regularization_data
#  A_layer_fraction, B_layer_fraction
#  prob_beta
#  probability_plus, B_plus, A_plus, Flow_prob_beta, Flow_Prob_Beta, Fraction_A_state, Fraction_B_state, fraction_A_layer, fraction_B_layer
#  MEAN_FLOW_PROB_BETA, PT, PB, PA, PAB, FA, FB, FAB, mean_Probability_plus, MEAN_Fraction_A_state, MEAN_Fraction_B_state
#  data_result, DATA_MFPBS, DATA_MFAS, DATA_MFBS, ganma, beta, prob_p, prob_q, Ganma, Beta, mean_PB, mean_PA, mean_PT, mean_PAB, 
#  mean_P_plus, Prob_P, Prob_Q, mean_FA, mean_FB, mean_FAB
   


In [252]:
class interconnected_A_layer_modeling :
    def __init__(self, state, node, edge, Max, Min, network) :   # network : 1 = random regular graph   2 = barabasi-albert graph
        self.state = state
        self.node = node
        self.edge = edge
        self.Max = Max
        self.Min = Min
        self.network = network
        
    def A_layer_config(self) :   # A_layer 구성요소 A_layer_config(state = [1,2], node = 2048, edge = 5, Max = 2, Min = -2)
        global A_state, A_node, A_edge, MAX, MIN
        A_state = self.state                                 # state = [   ]  kinds of states
        A_node = self.node
        A_edge = self.edge
        MAX = self.Max
        MIN = self.Min
        return A_state, A_node, A_edge, MAX, MIN
    
    def select_layer_A_model(self) :
        network = self.network
        if network == 1 :
            making_layer_A_random_regular()
        elif network == 2 :
            making_layer_A_barabasi_albert()
        return A, A_edges, MAX, MIN
    
def making_layer_A_random_regular() :               # A_layer random_regular network
    global A, A_edges
    A = np.array(A_state * int(A_node/len(A_state)), int)
    random.shuffle(A)
    A_edges = nx.random_regular_graph(A_edge, A_node, seed=None)
    return A, A_edges, MAX, MIN


def making_layer_A_barabasi_albert() :     # A_layer 바바라시-알버트 네트워크
    global A, A_edges
    A = np.array(A_state * int(A_node/len(A_state)), int)
    random.shuffle(A)
    A_edges = nx.barabasi_albert_graph(A_node, A_edge, seed=None)
    return A, A_edges, MAX, MIN

class interconnected_B_layer_modeling :
    def __init__(self, state, node, edge, inter_edges, network) :# network : 1 = random regular graph   2 = barabasi-albert graph
        self.state = state
        self.node = node
        self.edge = edge
        self.inter_edges = inter_edges
        self.network = network
        
    def B_layer_config(self) : # B_layer 구성요소 B_layer_config(state = [-1], node = 2048, edge = 5, inter_edge= 1)
        global B_state, B_node, B_edge, inter_edges
        B_state = self.state                                 # state = [   ]  kinds of states
        B_node = self.node
        B_edge = self.edge
        inter_edges = self.inter_edges  
        return B_state, B_node, B_edge, inter_edges

    def select_layer_B_model(self) :
        network = self.network
        if network == 1 :
            making_layer_B_random_regular()
        elif network == 2 :
            making_layer_B_barabasi_albert()
        return B, B_edges
    
    def making_interconnected_edges(self) :        # 
        global AB_edges, AB_neighbor
        inter_edges = self.inter_edges
        AB_edges = []
        AB_neighbor = []
        for i in range(int(A_node/inter_edges)) :
            for j in range(inter_edges) :
                connected_A_node = np.array(A_edges.nodes).reshape(-1, inter_edges)[i][j]
                AB_neighbor.append(connected_A_node)
                AB_edges.append((i, connected_A_node))
        AB_neighbor = np.array(AB_neighbor).reshape(-1, inter_edges)
        return AB_edges, AB_neighbor

def making_layer_B_random_regular() :               # B_layer random_regular network
    global B, B_edges
    B = np.array(B_state * int(B_node/len(B_state)), int)
    random.shuffle(B)
    B_edges = nx.random_regular_graph(B_edge, B_node, seed=None)
    return B, B_edges

def making_layer_B_barabasi_albert() :     # B_layer 바바라시-알버트 네트워크
    global B, B_edges
    B = np.array(B_state * int(B_node/len(B_state)), int)
    random.shuffle(B)
    B_edges = nx.barabasi_albert_graph(B_node, B_edge, seed=None)
    return B, B_edges

def simulation_condition(ganma_scale, beta_scale, Repeating_number, Limited_time) :
    global r, D, repeating_number, limited_time
    r = np.linspace(ganma_scale[0], ganma_scale[1], ganma_scale[2])
    D = np.linspace(beta_scale[0], beta_scale[1], beta_scale[2])
    repeating_number = Repeating_number            # 평균을 내기위한 반복 횟수
    limited_time = Limited_time                    # layer 변화를 위한 반복 횟수
    return r, D, repeating_number, limited_time

def variables_regularization(result,ganma_scale, beta_scale, Repeating_number, Limited_time, number) :
    global regularization_data
    simulation_condition(ganma_scale, beta_scale, Repeating_number, Limited_time)
    variable_scale(a, b, c, d, e, f)
    df = pd.read_pickle(result)
    regular_ganma = df['ganma'] / ganma_scale[1]
    regular_beta = df['beta'] / beta_scale[1]
    regular_data = pd.DataFrame({'re_ganma': regular_ganma, 're_beta' : regular_beta})
    regularization_data = pd.concat([df, regular_data], axis= 1)
    regularization_data.to_pickle('regular_result'+ str(number)+ '_data.pickle')

def A_layer_persuasion_function(a, b) :  # A layer 중에서 same orientation 에서 일어나는  변동 현상
    z = random.random()
    if z < prob_p :
        if a > 0 :
            a = A_layer_node_right(a, MAX)
            b = A_layer_node_right(b, MAX)
        elif a < 0 :
            a = A_layer_node_left(a, MIN)
            b = A_layer_node_left(b, MIN)
    elif z > prob_p :
        a = a
        b = b
    return a, b

def A_layer_node_left(a, MIN) :
    if a >= MIN : 
        if a == MIN :
            a = a
        elif a < 0 or a > 1 :
            a = a - 1
        elif a == 1 :
            a = -1
    elif a < MIN : 
        a = MIN
    return a

def A_layer 

def A_layer_node_right(a, MAX) :
    if a <= MAX : 
        if a == MAX :
            a = a
        elif a > 0 or a < -1 :
            a = a + 1
        elif a == -1 :
            a = 1
    elif a > MAX : 
        a = MAX
    return a

def A_layer_compromise_function(a, b) :  # A layer  중에서 opposite orientation 에서 일어나는 변동 현상 
    z = random.random()
    if z < (1-prob_p) :
        if a * b == -1 :
            if z < ((1-prob_p)/2) :
                a = 1
                b = 1
            elif z > ((1-prob_p)/2) :
                a = -1
                b = -1
        elif a > b :
            a = A_layer_node_left(a, MIN)
            b = A_layer_node_right(b, MAX)         
        elif a < b :
            a = A_layer_node_right(a, MAX)
            b = A_layer_node_left(b, MIN) 
    elif z > (1-prob_p) :
        a = a
        b = b
    return a, b

def AB_layer_persuasion_function(a, b) :  # A-B layer 중에서 same orientation 에서 일어나는  변동 현상
    z = random.random()
    if z < prob_p :
        if a > 0 :
            a = A_layer_node_right(a, MAX)
            b = b
        elif a < 0 :
            a = A_layer_node_left(a, MIN)
            b = b
    elif z > prob_p :
        a = a
        b = b
    return a, b

def AB_layer_compromise_function(a, b) :  # A-B layer  중에서 opposite orientation 에서 일어나는 변동 현상 
    z = random.random()
    if z < (1-prob_p) :
        if a * b == -1 :
            if z < ((1-prob_p)/2) :
                a = 1
                b = b
            elif z > ((1-prob_p)/2) :
                a = -1
                b = b
        elif a > b :
            a = A_layer_node_left(a, MIN)
            b = b      
        elif a < b :
            a = A_layer_node_right(a, MAX)
            b = b 
    elif z > (1-prob_p) :
        a = a
        b = b
    return a, b

        
def AS_model_function(a) :   # B layer 에서 일어나는 변동 현상
    z = random.random()
    if z < prob_beta :
        a = -a 
    return a

def fraction_different_state() :
    global A_layer_fraction, B_layer_fraction
    A_plus = sum(A > 0) 
    A_minus = sum(A < 0)
    if A_plus >= A_minus :
        A_layer_fraction = min(A_plus, A_minus) / len(A)
    elif A_plus < A_minus :
        A_layer_fraction = -(min(A_plus, A_minus)) / len(A)
    B_plus = sum(B > 0) 
    B_minus = sum(B < 0)    
    if B_plus >= B_minus :
        B_layer_fraction = min(B_plus, B_minus) / len(B)
    elif B_plus < B_minus :
        B_layer_fraction = -(min(B_plus, B_minus)) / len(B)
    return A_layer_fraction, B_layer_fraction
    
def A_layer_dynamics() :    # A_layer 다이내믹스, 감마 적용 및 설득/타협 알고리즘 적용
    for i, j in sorted(A_edges.edges()) :
        if A[i] * A[j] > 0 :
            A[i] = A_layer_persuasion_function(A[i], A[j])[0]
            A[j] = A_layer_persuasion_function(A[i], A[j])[1]
        elif A[i] * A[j] < 0 :
            A[i] = A_layer_compromise_function(A[i], A[j])[0]
            A[j] = A_layer_compromise_function(A[i], A[j])[1]
    for i, j in sorted(AB_edges) :    
        if A[j] * B[i] > 0 :
            A[j] = AB_layer_persuasion_function(A[j], B[i])[0]
            B[i] = AB_layer_persuasion_function(A[j], B[i])[1]
        elif A[j] * B[i] < 0 :
            A[j] = AB_layer_compromise_function(A[j], B[i])[0]
            B[i] = AB_layer_compromise_function(A[j], B[i])[1]
    return A, prob_p, prob_q

def B_layer_dynamics() :  # B_layer 다이내믹스, 베타 적용 및 언어데스 알고리즘 적용
    global prob_beta
    flow_prob_beta = []   # 베타 적용 확률 변화 리스트
    for i in sorted(B_edges.nodes) :
        opposite = []
        intra_edge_number = len(sorted(nx.all_neighbors(B_edges, i)))
        inter_edge_number = len(AB_neighbor[i])
        for j in range(intra_edge_number) :
            if B[i] * B[sorted(nx.all_neighbors(B_edges, i))[j]] < 0 :
                opposite.append(1)
        for j in range(inter_edge_number):
            if B[i] * A[AB_neighbor[i][j]] < 0 :
                opposite.append(1)
        prob_beta = (sum(opposite) / (inter_edge_number + intra_edge_number))**beta
        B[i] = AS_model_function(B[i])        
        flow_prob_beta.append(prob_beta)             # 베타 적용 확률 변화 리스트
    Flow_prob_beta.append(np.mean(flow_prob_beta))   #  노드의 prob_beta 평균에 대한 변화 리스트
    return B, Flow_prob_beta


def interconnected_dynamics() :
    global Probability_plus, B_plus, A_plus, Flow_prob_beta, Flow_Prob_Beta, Fraction_A_state, Fraction_B_state, fraction_A_layer, fraction_B_layer
    Flow_prob_beta = []                              #  노드의 prob_beta 평균에 대한 변화 리스트
    Flow_Prob_Beta = pd.DataFrame()                  #  prob_beta 평균의 변화를 칼럼으로 정리
    fraction_A_state = []                            #  A_layer의 different state 비율
    fraction_B_state = []                            #  B_layer의 different state 비율
    Fraction_A_state = pd.DataFrame()                #  A_layer의 different state 비율(데이터프레임)
    Fraction_B_state = pd.DataFrame()                #  B_layer의 different state 비율(데이터프레임)
    Probability_plus = 0                             # +로 consensus 되는 확률 측정
    total = 0
    while True :
        A_layer_dynamics()
        B_layer_dynamics()
        fraction_different_state()
        total += 1 
        fraction_A_state.append(A_layer_fraction)
        fraction_B_state.append(B_layer_fraction)
        if (np.all(A > 0) == 1 and np.all(B > 0) == 1) or (np.all(A < 0)== 1 and np.all(B < 0)== 1) or (total == limited_time) :
            break
    Flow_Prob_Beta = pd.DataFrame(Flow_prob_beta)  #  prob_beta 평균의 변화를 칼럼으로 정리
    Fraction_A_state = pd.DataFrame(fraction_A_state)  #  prob_beta 평균의 변화를 칼럼으로 정리
    Fraction_B_state = pd.DataFrame(fraction_B_state)  #  prob_beta 평균의 변화를 칼럼으로 정리
    if np.all(A > 0) == 1 and np.all(B > 0) == 1 :
        Probability_plus +=1
    elif np.all(A < 0)== 1 and np.all(B < 0)== 1 :
        Probability_plus +=0
    elif total == limited_time :
        Probability_plus +=0
    B_plus = sum(B)        # B_layer 노드의 합
    A_plus = sum(A)        # A_layer 노드의 합
    fraction_A_layer = (sum(A > 0) / len(A))
    fraction_B_layer = (sum(B > 0) / len(B))
    return A, B, A_plus, B_plus, passtime, Probability_plus, Flow_Prob_Beta, Fraction_A_state, Fraction_B_state, fraction_A_layer, fraction_B_layer

        
def repeated_and_mean() :
    global MEAN_FLOW_PROB_BETA, PT, PB, PA, PAB, FA, FB, FAB, mean_Probability_plus, MEAN_Fraction_A_state, MEAN_Fraction_B_state
    PB = []     # B_layer 노드의 평균값 리스트
    PA = []     # A_layer 노드의 평균값 리스트
    PAB = []
    FA = []
    FB = []
    FAB = []
    mean_Probability_plus = [] # +로 consensus 되는 확률 리스트
    MEAN_FLOW_Prob_Beta = pd.DataFrame()
    MEAN_FLOW_PROB_BETA = pd.DataFrame()
    MEAN_fraction_A_state = pd.DataFrame()
    MEAN_Fraction_A_state = pd.DataFrame()
    MEAN_fraction_B_state = pd.DataFrame()
    MEAN_Fraction_B_state = pd.DataFrame()
    for i in range(repeating_number) :
        AL.select_layer_A_model()
        BL.select_layer_B_model()
        BL.making_interconnected_edges()
        interconnected_dynamics() 
        PB.append(B_plus / B_node)
        PA.append(A_plus / A_node)
        PAB.append((B_plus / B_node) + (A_plus / A_node))
        FA.append(fraction_A_layer)
        FB.append(fraction_B_layer)
        FAB.append(fraction_A_layer+fraction_B_layer)
        mean_Probability_plus.append(Probability_plus)
        MEAN_FLOW_Prob_Beta = pd.concat([MEAN_FLOW_Prob_Beta, Flow_Prob_Beta], axis= 1, ignore_index = True)  # 반복횟수에 따라 칼럼병합
        MEAN_fraction_A_state = pd.concat([MEAN_fraction_A_state, Fraction_A_state], axis= 1, ignore_index = True)  # 반복횟수에 따라 칼럼병합
        MEAN_fraction_B_state = pd.concat([MEAN_fraction_A_state, Fraction_B_state], axis= 1, ignore_index = True)  # 반복횟수에 따라 칼럼병합
        
    MEAN_FLOW_PROB_BETA = MEAN_FLOW_Prob_Beta.fillna(0)    #빈칸 0으로 채우기   
    MEAN_Fraction_A_state = MEAN_fraction_A_state.fillna(0)  
    MEAN_Fraction_B_state = MEAN_fraction_B_state.fillna(0)
    return PB, PA, PAB, FA, FB, FAB, mean_Probability_plus, MEAN_FLOW_PROB_BETA, MEAN_Fraction_A_state, MEAN_Fraction_B_state

def making_dataframe() :    
    global data_result, DATA_MFPBS, DATA_MFAS, DATA_MFBS, ganma, beta, prob_p, prob_q, Ganma, Beta, mean_PB, mean_PA, mean_PT, mean_PAB, mean_P_plus, Prob_P, Prob_Q, mean_FA, mean_FB, mean_FAB
    mean_PB = []               # 반복횟수에 대한 B_layer 노드의 평균값
    mean_PA = []               # 반복횟수에 대한 A_layer 노드의 평균값
    mean_PAB = []
    mean_FA = []
    mean_FB = []
    mean_FAB = []
    mean_P_plus = []           # 반복횟수에 대한 +로 consensus 되는 확률의 평균값 리스트
    Ganma = []                 # ganma 리스트
    Beta = []
    Prob_P = []                # 확률 P 리스트
    Prob_Q = []                # 확률 Q 리스트
    data_MFPBS = pd.DataFrame()  
    Data_MFPBS = pd.DataFrame()
    DATA_MFPBS = pd.DataFrame() 
    data_MFAS = pd.DataFrame()  
    Data_MFAS = pd.DataFrame()
    DATA_MFAS = pd.DataFrame() 
    data_MFBS = pd.DataFrame()     
    Data_MFBS = pd.DataFrame()
    DATA_MFBS = pd.DataFrame() 
    for ganma in r :
        for beta in D :
            MFPBS = []
            MFAS = []
            MFBS = []
            prob_p = ganma / (1+ganma)   # 확률 p
            prob_q = 1 - prob_p          # 확률 q
            repeated_and_mean()
            Ganma.append(ganma)
            Beta.append(beta)
            Prob_P.append(prob_p)
            Prob_Q.append(prob_q)
            mean_PT.append(sum(PT) / repeating_number)
            mean_PB.append(sum(PB) / repeating_number)
            mean_PA.append(sum(PA) / repeating_number)
            mean_PAB.append(sum(PAB) / repeating_number)
            mean_FA.append(sum(FA) / repeating_number)
            mean_FB.append(sum(FB) / repeating_number)
            mean_FAB.append(sum(FAB) / repeating_number)
            mean_P_plus.append(sum(mean_Probability_plus) / repeating_number)
            MFPBS.append(ganma)
            MFPBS.append(beta)
            MFAS.append(ganma)
            MFAS.append(beta)
            MFBS.append(ganma)
            MFBS.append(beta)
            for i in range(len(MEAN_FLOW_PROB_BETA)) : 
                MFPB = np.mean(MEAN_FLOW_PROB_BETA.iloc[i])   # 반복횟수에 따른 베타확률의 평균값 정리
                MFPBS.append(MFPB)  
            data_MFPBS = pd.DataFrame(MFPBS)   # 반복횟수별 평균값으로 칼럼으로 정리
            Data_MFPBS = pd.concat([Data_MFPBS, data_MFPBS], axis= 1, ignore_index = True) # 감마,베타별 변화상태 정리
            
            for i in range(len(MEAN_Fraction_A_state)) : 
                MFA = np.mean(MEAN_Fraction_A_state.iloc[i])   # 반복횟수에 따른 different state fraction의 평균값 정리
                MFAS.append(MFA)  
            data_MFAS = pd.DataFrame(MFAS)   # 반복횟수별 평균값으로 칼럼으로 정리
            Data_MFAS = pd.concat([Data_MFAS, data_MFAS], axis= 1, ignore_index = True) # 감마,베타별 변화상태 정리

            for i in range(len(MEAN_Fraction_B_state)) : 
                MFB = np.mean(MEAN_Fraction_B_state.iloc[i])   # 반복횟수에 따른 different state fraction의 평균값 정리
                MFBS.append(MFB)  
            data_MFBS = pd.DataFrame(MFBS)   # 반복횟수별 평균값으로 칼럼으로 정리
            Data_MFBS = pd.concat([Data_MFBS, data_MFBS], axis= 1, ignore_index = True) # 감마,베타별 변화상태 정리
 

            if beta % 1 == 0 :
                print(ganma, beta, (sum(PT) / repeating_number), (sum(PB) / repeating_number), (sum(PA) / repeating_number), (sum(PAB) / repeating_number))
    data_result = pd.DataFrame({'ganma': Ganma, 'beta' : Beta, 'A layer mean' : mean_PA,'B layer mean' : mean_PB, 'A B layer mean' : mean_PAB,
                                 'Probability + mean' : mean_P_plus, 'Prob_P' :  Prob_P, 
                                 'Prob_Q' :  Prob_Q, 'Fraction A' : mean_FA, 'Fraction B' : mean_FB, 'Fraction AB' : mean_FAB})
    DATA_MFPBS = Data_MFPBS.fillna(0)   # 빈칸은 0으로 채우기
    DATA_MFAS = Data_MFAS.fillna(0)   # 빈칸은 0으로 채우기
    DATA_MFBS = Data_MFBS.fillna(0)   # 빈칸은 0으로 채우기
    return data_result, DATA_MFPBS, DATA_MFAS, DATA_MFBS

def saving_data(a) :
    Total_data.to_pickle('result'+ str(a)+ '_data.pickle')
    DATA_MFPBS.to_pickle('flow_prob_beta'+str(a)+ '_data.pickle')
    DATA_MFAS.to_pickle('A_different_state'+str(a)+ '_data.pickle')
    DATA_MFBS.to_pickle('B_different_state'+str(a)+ '_data.pickle')
    
def plot_3D(result, a, b, c, d, e, f) : # ex__  plot_3D('result11_data2.pickle', 'ganma', 'beta', 'B layer mean', 45, 45, 'A b layer mean')
    sns.set_style("whitegrid")
    final_data = pd.read_pickle(result)
    ax = plt.axes(projection='3d')
    ax.plot_trisurf(final_data[a], final_data[b], final_data[c], cmap='RdBu', edgecolor='none')
    ax.set_xlabel(str(a))
    ax.set_ylabel(str(b))
    ax.set_zlabel(str(c))
    ax.set_title(str(f))
    ax.view_init(d, e)
    # ax.scatter(final_data[a], final_data[b], final_data[c], c=final_data[c], cmap='RdBu', linewidth=0.1) ax.view_init(d, e)

def plot_2D(result, a, b, c, d, e) :       # ex__  plot_2D('result2.3_data.pickle', 'beta', 'B layer mean', 'ganma', 1.0, 'ganma = 1')
    sns.set_style("whitegrid")
    final_data = pd.read_pickle(result)
    mini = final_data[final_data[c] > (d - 0.01)]
    two_dimension = mini[mini[c] < (d + 0.01)]
    plt.plot(two_dimension[a], two_dimension[b], '-', label = e)
    plt.legend(framealpha = 1, frameon = True)
    plt.ylim(-1.1, 1.1)
    plt.xlabel(str(a))
    plt.ylabel(str(b))

def pandas_concat1(a, b, c, d, e) : # pandas-concat('결과1', '결과2', '저장할 이름')
    result_1 = pd.read_pickle(a)
    result_2 = pd.read_pickle(b)
    result_3 = pd.read_pickle(c)
    result_4 = pd.read_pickle(d)
    final_result = pd.concat([result_1, result_2, result_3, result_4], ignore_index = True)
    final_result_dropped = final_result.drop_duplicates(['ganma', 'beta'], keep = 'first')
    final_result_dropped.to_pickle(e)  
    
def pandas_concat2(a, b, c, d, e) : # pandas-concat('결과1', '결과2', '저장할 이름')
    result_1 = pd.read_pickle(a)
    result_2 = pd.read_pickle(b)
    result_3 = pd.read_pickle(c)
    result_4 = pd.read_pickle(d)
    final_result1 = pd.concat([result_1, result_2], axis = 1, ignore_index = True)
    final_result2 = pd.concat([result_3, result_4], axis = 1, ignore_index = True)
    final_result3 = pd.concat([final_result1, final_result2], axis = 1, ignore_index = True)
    final_result3.to_pickle(e)
    
def different_state_ratio(filename, time, new_filename) :   # different state ratio로 편집해주는 함수
    reading_file = pd.read_pickle(filename)
    index_g = (reading_file.iloc[0, :]) ## ganma index 지정
    index_b = reading_file.iloc[1, :]   ## beta index 지정
    index_t = []                        ## time index 지정
    for i in range(0, time) : 
        index_t.append(i) 
    different_state = reading_file.iloc[2:, :]
    dic = {}
    for i in range(len(index_g)) :
        for j in range(0, time) :
            data = different_state.iloc[j, i]
            dic[index_g[i], index_b[i], j] = data
    frame = pd.Series(dic)
    frame.index.names = ['ganma', 'beta', 'time']
    df = frame.reset_index(name = 'Fraction Different')
    DF = df.set_index(['ganma', 'beta', 'time'])
    final_table = DF.pivot_table('Fraction Different', 'time', ['beta', 'ganma'])
    final_table.to_pickle(new_filename)
    return final_table

    
def prob_beta_plot_3D(result, number_ganma, t, initial, gap, a, b, c, d, e) : 
    flow_probbeta_data = pd.read_pickle(result)
    ganma_data = []
    probbeta = []
    times = np.linspace(0,t-1,t)
    time = sorted(sorted(times) * number_ganma)
    for i in range(number_ganma) :
        ganma_data.append(flow_probbeta_data.iloc[0, initial+(gap*i)])
    ganma_datas = (ganma_data) * t
    for j in range(t) :
        for i in range(number_ganma) :
            probbeta.append(flow_probbeta_data.iloc[2+j, initial+(gap*i)])
    sns.set_style("whitegrid")
    ax=plt.axes(projection='3d')
    ax.plot_trisurf(time, ganma_datas, probbeta, cmap='viridis', edgecolor='none')
    ax.set_xlabel(str(a))
    ax.set_ylabel(str(b))
    ax.set_zlabel(str(c))
    ax.view_init(d, e)
# ex__  prob_beta_plot_3D('flow_prob_beta5.0_data.pickle', 51, 20, 10, 101, 'time', 'ganma', 'prob_beta', 45, 45)

def total_flow_prob_beta_chart(filename) : 
    plt.figure()
    sns.set()
    da = pd.read_pickle(filename)
    plt.plot(da, linewidth = 0.2)
    plt.ylabel('probability for layer B')
    plt.xlabel('time(step)')

def total_different_state_ratio_chart(filename, ylabel) : 
    plt.figure()
    sns.set()
    da = pd.read_pickle(filename)
    plt.plot(da, linewidth = 0.2)
    plt.ylabel(ylabel)
    plt.xlabel('time(step)')
    
def beta_scale_for_chart(filename,y_axis, a, b) :   # 0 < a, b < 3  
    plt.figure()
    sns.set()
    da = pd.read_pickle(filename)
    plt.ylim(-0.5, 0.5)
    plt.ylabel(y_axis)
    plt.xlabel('time(step)')
    beta_scale = da.columns.levels[0] 
    min_beta = sum(beta_scale < a)   
    max_beta = sum(beta_scale < b) 
    for i in range(min_beta, max_beta) :
        pic = da[beta_scale[i]]
        plt.plot(pic, linewidth = 0.3)    
        
def ganma_scale_for_chart(filename,y_axis, a, b) :   # 0 < a, b < 3  
    plt.figure()
    sns.set()
    da = pd.read_pickle(filename)
    unstack = da.unstack()
    reset = unstack.reset_index(name='different_state')
    Reset = pd.DataFrame(reset)
    final_table =Reset.pivot_table('different_state', 'time', ['ganma', 'beta'])
    plt.ylim(-0.5, 0.5)
    plt.ylabel(y_axis)
    plt.xlabel('time(step)')
    ganma_scale = final_table.columns.levels[0] 
    min_beta = sum(ganma_scale < a)   
    max_beta = sum(ganma_scale < b) 
    for i in range(min_beta, max_beta) :
        pic = final_table[ganma_scale[i]]
        plt.plot(pic, linewidth = 0.3)   

def z_function(result, a):
    final_data = pd.read_pickle(result)
    z = np.array(final_data[str(a)]).reshape(41, 41)
    Z = np.zeros((1681, 1681))
    for i in range(0, 41) :
            for j in range(0, 41) :
                for k in range(0, 41) :
                    for l in range(0, 41) :
                        Z[(i*41)+k][(j*41)+l] = z[i][j]
    return Z

#plot_3D_to_2D_contour('result128128_1(5)_1(5)ver2_data.pickle', 'beta','ganma', 'A B layer mean','beta', 'ganma', 'A B layer mean')
def plot_3D_to_2D_contour(result, a, b, c, d, e, f) : 
    sns.set_style("whitegrid")
    final_data = pd.read_pickle(result)
    result_a = sorted(np.array(final_data[str(a)]))
    result_b = sorted(np.array(final_data[str(b)]))
    result_c = np.array(final_data[str(c)]).reshape(41, 41)
    X, Y = np.meshgrid(result_a, result_b)
    Z = z_function(result, c)
    plt.contourf(X, Y, Z, 50, cmap = 'RdBu')
    plt.xlabel(str(d))
    plt.ylabel(str(e))
    plt.colorbar(label = str(f))
    
def making_beta_scale(a) :
    scale = math.log((1/(B_edge + 1))**3) / math.log(inter_edges / (B_edge +inter_edges))
    return(0, scale, a)